In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
training_dataset = pd.read_csv('train 2.csv')
testing_dataset = pd.read_csv('test 2.csv')

In [ ]:
training_dataset.head()

In [ ]:
testing_dataset.head()

In [ ]:
X_train = training_dataset.drop('id', axis=1)
X_train = X_train.drop('Exited', axis=1)
X_train = X_train.drop('CustomerId', axis=1)
X_train = X_train.drop('Surname', axis=1)
y_train = training_dataset['Exited']

X_test = testing_dataset.drop('id', axis=1)
X_test = X_test.drop('CustomerId', axis=1)
X_test = X_test.drop('Surname', axis=1)
label = testing_dataset.iloc[:, 0]

In [ ]:
y_train

In [ ]:
X_test.head()

In [ ]:
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

In [ ]:
numerical_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

In [ ]:
X_train_transform = preprocessor.fit_transform(X_train)
X_test_transform = preprocessor.transform(X_test)

In [ ]:
param_grid = {
    'C': [0.1, 1],
    'gamma': [1, 0.1],
    'kernel': ['rbf', 'poly', 'sigmoid'] 
}

In [ ]:
model = SVC(kernel='rbf', C=1, gamma=0.01)
model.fit(X_train_transform, y_train)

In [ ]:
y_pred = model.predict(X_test_transform)

In [ ]:
result_df = pd.DataFrame({
    'id': label,
    'Exited': y_pred
})

result_df.to_csv('predictions.csv', index=False)